In [0]:
pip install geopandas

In [0]:
%python

import pandas as pd
from datetime import datetime
import geopandas as gpd
from shapely import wkt, union_all, intersection


In [0]:
pigeons = pd.read_csv('raw/pigeonnier_update.csv',sep=';',skiprows=1)
pigeons.columns = ['date','pedestrians','debug','dropped']
pigeons.date = pigeons['date'].apply(lambda x : datetime.strptime(x, "%Y-%m-%d %H:%M:%S").date())

In [0]:
%python

pigeons[pigeons.pedestrians>0][['date','pedestrians']].head(2)

# Outdoor Vision : Outings

In [0]:
ov_file = "raw/CC_Gorges_Ardeche_2020_2024_0.gpkg"
outdoor_vision = gpd.read_file(ov_file)

In [0]:
#outdoor_vision.provider.drop_duplicates()
#outdoor_vision.activity.drop_duplicates()
red_outdoor = outdoor_vision[['activity','user_id','date_start','date_end','geometry']]
red_outdoor.head(1)

In [0]:
red_outdoor.date_start.min()

In [0]:
red_outdoor.shape

# Biodiv : Sensitive Areas

In [0]:
biodiv = gpd.read_file("https://object.files.data.gouv.fr/dataeng-open/hackathon_outdoor/SIG/03-Biodiversite/Data_biodiv/data_geonatureSGGA.geojson")

In [0]:
biodiv['species_age'] = 'To Fill'
biodiv.loc[biodiv.stade_vie.isin(['Imago','Adulte']),'species_age'] = 'Adulte'
biodiv.loc[biodiv.stade_vie.isin(['Inconnu','Indéterminé',None]),'species_age'] = 'Inconnu'
biodiv.loc[biodiv.stade_vie.isin(['Exuvie','Larve','Juvénile','Œuf']),'species_age'] = 'Juvénile'
biodiv.loc[biodiv.stade_vie.isin(['Immature']),'species_age'] = 'Immature'

biodiv['behaviour'] = 'To Fill'
biodiv.loc[biodiv.comportement.isin(['Inconnu','Non renseigné',None]),'behaviour'] = 'Inconnu'
biodiv.loc[biodiv.comportement.isin(['Chasse/alimentation','Déplacement','Passage en vol','Hivernage','Repos','Halte migratoire']),'behaviour'] = 'Quotidien'
biodiv.loc[biodiv.comportement.isin(['Accouplement','Territorial','Tandem','Pond','Chant','Dortoir']),'behaviour'] = 'Territorialité'

In [0]:
to_keep = ['cd_nom', 'group2_inpn', 'niveau_sensibilite',
           'nom_valide', 
           'date_debut', 'date_fin', 
           'behaviour', 'species_age', 
           #'nombre_max', 'nombre_min', 'champs_additionnels',
           'geometry'
           # ,'x_centroid_4326', 'y_centroid_4326', 'geometrie_wkt_4326'
           ]
cleaned_biodiv = biodiv[(biodiv.regne == 'Animalia')
                        &(biodiv.date_debut >= '2020-01-01')
                        &(biodiv.date_debut <= '2024-12-31')][to_keep]
cleaned_biodiv.head(3)

In [0]:
cleaned_biodiv['geometry'] = cleaned_biodiv['geometry'].buffer(1)

In [0]:
to_group = ['cd_nom', 'nom_valide', 'group2_inpn', 'niveau_sensibilite', 'date_debut', 'date_fin', 
            'behaviour', 'species_age']
biodiv_agg = cleaned_biodiv.groupby(to_group).agg({'geometry':lambda x: list(x)}).reset_index()
biodiv_agg.head(3)

In [0]:
biodiv_agg = biodiv_agg.groupby(to_group).apply(lambda x: union_all(list(x.geometry)))\
                     .reset_index(name='geometry')
biodiv_agg = gpd.GeoDataFrame(biodiv_agg, crs="EPSG:3857")

In [0]:
biodiv_agg.head(1)

In [0]:
biodiv_agg.to_file('processed/biodiv_agg.geojson', driver="GeoJSON")
biodiv_agg.to_csv('processed/biodiv_agg.csv', index=False)  

In [0]:
biodiv_agg.head(1)

# Intersects

In [0]:
in_bs = df_agg_bs.join(red_outdoor.head(50), lsuffix='_bs', rsuffix='_otd', how='cross')
in_bs['intersects'] = in_bs.apply(lambda x: x.geometry_bs.intersects(x.geometry_otd), axis=1)
in_bs = in_bs[in_bs['intersects']==True]#[['nom_cite','activity','user_id','date_start','date_end',]]

In [0]:
in_bs.head(1)

In [0]:
import matplotlib.pyplot as plt

df_plot = gpd.GeoDataFrame()
df_plot['geometry'] = [df_all_agg[df_all_agg['otd_exp_id'] == '62815f0535238'].reset_index()[['geometry_bs']].iloc[0, 0],
                       df_all_agg[df_all_agg['otd_exp_id'] == '62815f0535238'].reset_index()[['geometry_otd']].iloc[0, 0]]
df_plot['color'] = ['green', 'red']
df_plot = df_plot.set_geometry('geometry')

df_plot.plot(figsize=(8, 8), color=df_plot['color'])
plt.show()